In [ ]:
pip install accelerate

In [ ]:
pip install peft

In [ ]:
pip install tokenizer

In [1]:
import tokenizer
import torch
import peft
import accelerate

In [ ]:
pip install transformers

In [2]:
import gc

def cleanup():
    if 'model' in globals():
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect()
    torch.cuda.empty_cache()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

In [4]:
import torch

In [5]:
def load_model_and_tokenizer(model_id, peft=None):

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16,  device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=[
                                        "q_proj", "k_proj", "v_proj", "o_proj",
                                        "gate_proj", "up_proj", "down_proj"
                                    ],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

In [6]:
import torch
def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요")

In [7]:
model_id = "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"

In [8]:
model, tokenizer = load_model_and_tokenizer(model_id, peft='lora')

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/835M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

trainable params: 15,196,160 || all params: 4,326,660,878 || trainable%: 0.3512
GPU 메모리 사용량: 8.088 GB


In [9]:
tokenizer("심장이 매우 뛰어요")

{'input_ids': [128000, 102612, 114784, 121520, 123151, 105807], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [10]:
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 15196160
all params: 4326660878
trainable%: 0.3512214252165848


In [14]:
from datasets import load_dataset

In [15]:
data = load_dataset("csv", data_files={"train" : "./train_data.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
data2 = data.map(lambda x : {'text' : f"""User: {x['question']} 
                    Assistant: {x['answer']}<|endoftext|>"""})

Map:   0%|          | 0/12228 [00:00<?, ? examples/s]

In [17]:
for x in data2['train']:
    print(x['text'])
    break

User: 10세 소년이 충치 치료를 위해 치과에 방문했다. 치과의사는 수복 재료로 아말감 대신 복합 레진을 권장하고 있다. 복합 레진 사용의 장점으로 가장 적절한 것은?  
1) 심미성이 좋다.  
2) 수복물의 강도가 높다.  
3) 수은 함유로 인한 독성 위험이 있다.  
4) 비용이 저렴하다.  
5) 이차 우식증 발생 위험이 낮다. 
                    Assistant: 1) 심미성이 좋다.<|endoftext|>


In [18]:
from transformers import TrainingArguments, Trainer

In [19]:
argu = TrainingArguments(
         per_device_train_batch_size=1,
        num_train_epochs=2, 
        gradient_accumulation_steps=4,
        learning_rate = 0.0001,
        fp16=True,
        output_dir="./output/"
    )

In [20]:
train_dataset = data2.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/12228 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorForLanguageModeling

In [22]:
trainer = Trainer(
    model = model,
    train_dataset=train_dataset['train'],
    args=argu,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
trainer.train()

Step,Training Loss
500,1.401800
1000,1.245800
1500,1.211800
2000,1.182900
2500,1.147100
3000,1.124600
3500,0.988100
4000,0.989200
4500,0.973700
5000,0.966000


TrainOutput(global_step=6114, training_loss=1.0922917299579475, metrics={'train_runtime': 4420.9497, 'train_samples_per_second': 5.532, 'train_steps_per_second': 1.383, 'total_flos': 9.5783623801107e+16, 'train_loss': 1.0922917299579475, 'epoch': 2.0})

In [24]:
# 치주 질환·충치 예방을 위한 우선 권장 구강 관리법 AI 생성 함수(코드 분석)

prompt = """질문에 해당되는 답변의 번호를 알려주세요
"""

from transformers.generation import GenerationConfig
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_special_tokens=True)  # 모델이 생성한 토큰을 실시간 출력(특수 토큰 제외)
streamer = TextStreamer(tokenizer)                            # 스트리머 객체(토크나이저 연결, 두 번 정의됨—실행상 영향은 없음)
gen_cfg = GenerationConfig.from_model_config(model.config)    # 현재 모델 설정 기반의 생성 파라미터 객체 생성
gen_cfg.pad_token_id = tokenizer.eos_token_id                 # 패딩 토큰 id를 eos_token_id로 지정

def gen(x):
    gen_cfg.temperature = 0.3                                 # 생성 다양성(랜덤성) 감소해 신뢰도 높임
    gen_cfg.max_new_tokens = 128                              # 최대 생성 토큰(문장 길이) 제한
    rt = tokenizer(
            f"User: {prompt}\n\n질문: {x}\n\nAssistant:",      # 사용자 프롬프트에 직접 질문과 입력값(x) 결합
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda')                                          # 텐서를 GPU로 전달

    gened = model.generate(
        **rt,                                                 # 입력 텐서(문장 변환)
        do_sample=True,                                       # 샘플링(확률적 생성) 활성화
        generation_config=gen_cfg,                            # 위에서 정의한 생성 파라미터 적용
        streamer=streamer,                                    # 실시간 토큰 스트리밍 활성
    )

    result = tokenizer.decode(gened[0])                       # 숫자 시퀀스를 한글 텍스트로 변환
    answer = result.split('Assistant:')                       # AI 답변 기준으로 문장 분리

    # return answer[1].strip()                                # 필요시 'Assistant:' 이후 실제 답변만 반환
    return result                                             # 전체 생성 결과 반환

query = """50세 남성이 최근 치과 검진에서 잇몸 출혈과 치아 사이의 플라그 축적으로 진단받았습니다. 치주 질환과 충치를 예방하기 위해 가장 우선적으로 권장되는 구강 관리 방법은 무엇입니까?  
1) 하루에 두 번 칫솔질하기  
2) 정기적인 스케일링 받기  
3) 구강 세정제 사용하기  
4) 치실 사용하기  
5) 치간 칫솔 사용하기"""


In [25]:
gen(query)

<|begin_of_text|>User: 질문에 해당되는 답변의 번호를 알려주세요


질문: 50세 남성이 최근 치과 검진에서 잇몸 출혈과 치아 사이의 플라그 축적으로 진단받았습니다. 치주 질환과 충치를 예방하기 위해 가장 우선적으로 권장되는 구강 관리 방법은 무엇입니까?  
1) 하루에 두 번 칫솔질하기  
2) 정기적인 스케일링 받기  
3) 구강 세정제 사용하기  
4) 치실 사용하기  
5) 치간 칫솔 사용하기

Assistant: 2) 정기적인 스케일링 받기<|endoftext|>정기적인 스케일링은 치주 질환과 충치를 예방하는 데 가장 효과적인 방법입니다. 스케일링은 치아 사이의 플라그를 제거하여 치주 질환의 발생을 줄이고, 치아의 건강을 유지하는 데 도움을 줍니다. 따라서 정기적인 스케일링은 치과 방문 시 가장 중요한 관리 방법 중 하나입니다. 
                    Assistant: 2) 정기적인 스케일링 받기<|endoftext


'<|begin_of_text|>User: 질문에 해당되는 답변의 번호를 알려주세요\n\n\n질문: 50세 남성이 최근 치과 검진에서 잇몸 출혈과 치아 사이의 플라그 축적으로 진단받았습니다. 치주 질환과 충치를 예방하기 위해 가장 우선적으로 권장되는 구강 관리 방법은 무엇입니까?\xa0\xa0\n1) 하루에 두 번 칫솔질하기\xa0\xa0\n2) 정기적인 스케일링 받기\xa0\xa0\n3) 구강 세정제 사용하기\xa0\xa0\n4) 치실 사용하기\xa0\xa0\n5) 치간 칫솔 사용하기\n\nAssistant: 2) 정기적인 스케일링 받기<|endoftext|>정기적인 스케일링은 치주 질환과 충치를 예방하는 데 가장 효과적인 방법입니다. 스케일링은 치아 사이의 플라그를 제거하여 치주 질환의 발생을 줄이고, 치아의 건강을 유지하는 데 도움을 줍니다. 따라서 정기적인 스케일링은 치과 방문 시 가장 중요한 관리 방법 중 하나입니다. \n                    Assistant: 2) 정기적인 스케일링 받기<|endoftext'